<a href="https://colab.research.google.com/github/cursosLabra/pruebawebsem/blob/main/Introducci%C3%B3n_a_Web_Sem%C3%A1ntica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a RDF

En esta sección, se va a introducir RDF. Para ello, se requiere instalar la  librería [rdflib](https://rdflib.readthedocs.io/).

In [ ]:
!pip install rdflib

: 

A partir de ahí, se pueden crear grafos importando las clases correspondientes de rdflib.

In [ ]:
from rdflib import Graph

: 

In [ ]:
g = Graph()

: 

In [ ]:
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/example1.ttl", format="turtle")

: 

In [ ]:
for contador, t in enumerate(g):
  print(f"Tripleta {contador}: sujeto = {t[0]}, predicado = {t[1]}, objeto = {t[2]}") 

: 

In [ ]:
print(g.serialize(format="nt"))

: 

In [ ]:
print(g.serialize(format="xml"))

: 

In [ ]:
print(g.serialize(format="json-ld"))

: 

In [ ]:
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/example2.ttl", format="turtle")
print(g.serialize(format="turtle"))

: 

In [ ]:
g1.

: 